# Compute the trend of a time series 

Let's perform a linear regression on time series data by exploiting *PyOphidia*, *numpy* and *matplotlib* for the visualization. The goal is to extract the trend of the time series.

1- First of all import the PyOphidia modules and connect to the server-side

In [ ]:
from PyOphidia import cube
cube.Cube.setclient(read_env=True)

2- Load the *tasmin* variable from the input dataset located at */public/data/ecas_training/tasmin_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc*. The data is imported so that it is time-array oriented.

In [ ]:
mintemp = cube.Cube.importnc(
    src_path='/data/tasmin_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc',
    measure='tasmin',
    import_metadata='yes',
    imp_dim='time',
    imp_concept_level='d', vocabulary='CF',hierarchy='oph_base|oph_base|oph_time',
    ncores=2,
    description='Min Temps')

3- Extract a single time series from the imported cube *(just consider the first index for latitute and longitude)* and print its content

In [ ]:
singleTS = mintemp.subset(
    subset_dims="lat|lon",
    subset_filter="1|1",
    subset_type='index',
    ncores=2)

singleTS.explore()

4- Run the Ophidia primitive to perform the linear regression over the time series.
The *apply* operator with the *oph_gsl_fit_linear* primitive can be used.

**NOTE:** the resulting cube will just provide two values, the intercept and the slope.

In [ ]:
trendCube = singleTS.apply(
    query="oph_gsl_fit_linear_coeff('OPH_FLOAT','OPH_FLOAT',measure, dimension,'110000')",
    check_type='no')

5- Let's plot the data with respect to the time dimensions (indexes), as well as the linear regression line in a scatter plot. Note that:

```python
trendCoeff = trendCube.export_array(show_time='no')
intercept = trendCoeff['measure'][0]['values'][0][0]
slope = trendCoeff['measure'][0]['values'][0][1]
```

In [ ]:
%matplotlib inline

data = singleTS.export_array(show_time='no')
trendCoeff = trendCube.export_array(show_time='no')

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
import numpy as np

y = data['measure'][0]['values'][0][:]
x = data['dimension'][2]['values'][:]
fig, ax = plt.subplots(figsize=(11, 3), dpi=100)
ax.scatter(x, y)

intercept = trendCoeff['measure'][0]['values'][0][0]
slope = trendCoeff['measure'][0]['values'][0][1]

line = intercept + slope * np.array(x)
ax.plot(x,line, color='red')

plt.ylabel(data['measure'][0]['name'] + " (degK)")
plt.xlabel("Days since 2096/01/01")
plt.title('Minimum Near-Surface Air Temperature')
plt.show()


Finally clear your workspace from the datacubes produced during the experiments.

In [ ]:
cube.Cube.deletecontainer(container='tasmin_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc',force='yes')